In [13]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import xesmf as xe
import cartopy
import dask
from tqdm.autonotebook import tqdm  # Fancy progress bars for our loops!
import intake
import fsspec
import requests
import aiohttp
import nc_time_axis
import cftime

In [14]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = 12, 6

In [15]:
var_df = pd.read_csv("Financial_Support_Agreement_NEW.csv")
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,ps,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
1,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rsds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
2,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlus,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
3,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
4,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,psl,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706


In [16]:
# df_ta_ncar = df.query("activity_id=='CMIP' & table_id == 'Amon' &  source_id == 'CESM2' & variable_id == 'tas'  & institution_id == 'NCAR'")
df_ta_ncar = df.query("experiment_id == 'piControl' & table_id == 'Amon' &  source_id == 'CESM2' & activity_id=='CMIP' & institution_id == 'NCAR'")
vars_here = df_ta_ncar["variable_id"].copy()
# df_ta_ncar.experiment_id.unique()
display(df_ta_ncar)
# display(vars_here)
print(var_df[var_df["variable_id"].isin(vars_here)])

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
67548,CMIP,NCAR,CESM2,piControl,r1i1p1f1,Amon,rtmt,gn,gs://cmip6/CMIP6/CMIP/NCAR/CESM2/piControl/r1i...,NaN,20190320
67549,CMIP,NCAR,CESM2,piControl,r1i1p1f1,Amon,rsutcs,gn,gs://cmip6/CMIP6/CMIP/NCAR/CESM2/piControl/r1i...,NaN,20190320
67550,CMIP,NCAR,CESM2,piControl,r1i1p1f1,Amon,rsut,gn,gs://cmip6/CMIP6/CMIP/NCAR/CESM2/piControl/r1i...,NaN,20190320
67551,CMIP,NCAR,CESM2,piControl,r1i1p1f1,Amon,rsuscs,gn,gs://cmip6/CMIP6/CMIP/NCAR/CESM2/piControl/r1i...,NaN,20190320
67552,CMIP,NCAR,CESM2,piControl,r1i1p1f1,Amon,rsus,gn,gs://cmip6/CMIP6/CMIP/NCAR/CESM2/piControl/r1i...,NaN,20190320
67553,CMIP,NCAR,CESM2,piControl,r1i1p1f1,Amon,tauu,gn,gs://cmip6/CMIP6/CMIP/NCAR/CESM2/piControl/r1i...,NaN,20190320
67555,CMIP,NCAR,CESM2,piControl,r1i1p1f1,Amon,clt,gn,gs://cmip6/CMIP6/CMIP/NCAR/CESM2/piControl/r1i...,NaN,20190320
67556,CMIP,NCAR,CESM2,piControl,r1i1p1f1,Amon,cli,gn,gs://cmip6/CMIP6/CMIP/NCAR/CESM2/piControl/r1i...,NaN,20190320
67558,CMIP,NCAR,CESM2,piControl,r1i1p1f1,Amon,sfcWind,gn,gs://cmip6/CMIP6/CMIP/NCAR/CESM2/piControl/r1i...,NaN,20190320
67572,CMIP,NCAR,CESM2,piControl,r1i1p1f1,Amon,ta,gn,gs://cmip6/CMIP6/CMIP/NCAR/CESM2/piControl/r1i...,NaN,20190320


     variable_id                                        description  \
97           cli               Mass Fraction of Cloud Ice [kg kg-1]   
110        clivi                            Ice Water Path [kg m-2]   
114           cl                         Percentage Cloud Cover [%]   
120          clt                   Total Cloud Cover Percentage [%]   
122          clw      Mass Fraction of Cloud Liquid Water [kg kg-1]   
126        clwvi                      Condensed Water Path [kg m-2]   
132          co2                   Mole Fraction of CO2 [mol mol-1]   
250      evspsbl  Evaporation Including Sublimation and Transpir...   
371         hfls            Surface Upward Latent Heat Flux [W m-2]   
379         hfss          Surface Upward Sensible Heat Flux [W m-2]   
386          hur                              Relative Humidity [%]   
389         hurs                 Near-Surface Relative Humidity [%]   
391         huss                 Near-Surface Specific Humidity [1]   
392   

In [12]:
# df_ta_ncar = df.query("activity_id=='CMIP' & table_id == 'Amon' &  source_id == 'CESM2' & variable_id == 'tas'  & institution_id == 'NCAR'")
df_ta_ncar = df.query("experiment_id == 'historical' & table_id == 'Amon' &  source_id == 'CESM2' & activity_id=='CMIP'")
vars_here = df_ta_ncar["variable_id"].copy()
# df_ta_ncar.experiment_id.unique()
display(df_ta_ncar.head())
display(var_df[var_df["variable_id"].isin(vars_here)]["description"])

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
59364,CMIP,NCAR,CESM2,historical,r1i1p1f1,Amon,ch4global,gn,gs://cmip6/CMIP6/CMIP/NCAR/CESM2/historical/r1...,NaN,20190308
59373,CMIP,NCAR,CESM2,historical,r1i1p1f1,Amon,cfc11global,gn,gs://cmip6/CMIP6/CMIP/NCAR/CESM2/historical/r1...,NaN,20190308
59374,CMIP,NCAR,CESM2,historical,r1i1p1f1,Amon,cfc12global,gn,gs://cmip6/CMIP6/CMIP/NCAR/CESM2/historical/r1...,NaN,20190308
59382,CMIP,NCAR,CESM2,historical,r1i1p1f1,Amon,rlds,gn,gs://cmip6/CMIP6/CMIP/NCAR/CESM2/historical/r1...,NaN,20190308
59383,CMIP,NCAR,CESM2,historical,r1i1p1f1,Amon,rldscs,gn,gs://cmip6/CMIP6/CMIP/NCAR/CESM2/historical/r1...,NaN,20190308


58             Global Mean Mole Fraction of CFC11 [1e-12]
60             Global Mean Mole Fraction of CFC12 [1e-12]
63               Global Mean Mole Fraction of CH4 [1e-09]
81         Fraction of Time Convection Occurs in Cell [1]
97                   Mass Fraction of Cloud Ice [kg kg-1]
110                               Ice Water Path [kg m-2]
114                            Percentage Cloud Cover [%]
120                      Total Cloud Cover Percentage [%]
122         Mass Fraction of Cloud Liquid Water [kg kg-1]
126                         Condensed Water Path [kg m-2]
131                    Total Atmospheric Mass of CO2 [kg]
132                      Mole Fraction of CO2 [mol mol-1]
250     Evaporation Including Sublimation and Transpir...
371               Surface Upward Latent Heat Flux [W m-2]
379             Surface Upward Sensible Heat Flux [W m-2]
386                                 Relative Humidity [%]
389                    Near-Surface Relative Humidity [%]
391           